In [ ]:
# install depencies
!pip install datasets transformers

In [ ]:
import transformers, datasets

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!apt install git-lfs

In [ ]:
import transformers

print(transformers.__version__)

4.28.1


In [ ]:
from transformers.utils import send_example_telemetry
send_example_telemetry('language_modeling_notebook', framework='pytorch')

In [ ]:
from datasets import load_dataset
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

In [ ]:
dataset['train'][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML 

def show_random_elements(dataset, num_examples=10) :
  assert num_examples <= len(dataset), "Can't pich more elements than there are in the dataset"

  picks=[]

  for _ in range(num_examples) :
    pick = random.randint(0, len(dataset)-1)
    while pick in picks : 
      pick = random.randint(0, len(dataset)-1)
    picks.append(pick)

  df = pd.DataFrame(dataset[picks])

  for col, typ in dataset.features.items() :
    if isinstance(typ, ClassLabel) :
      df[column] = df[column].transform(lambda i: typ.names[i])

  display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,text
0,"The churchyard has seven gravestones that were discovered in the walls of the previous church . Six of them date from between the 9th and 11th centuries , and the seventh is from the 12th or 13th century . The doorway from the old church , dating from the 15th century , has been reused as an entrance to the churchyard . A carved stone human head , from the 12th century , has been inserted into the north wall of the churchyard . A war memorial in the shape of a Celtic cross remembers those who died during the First and Second World Wars . \n"
1,= = = 1993 – 1997 = = = \n
2,"[ T ] hose of high rank or superior in horsemanship wear gilded helmets of iron or bronze to draw the attention of the spectators . Unlike the helmets made for active service , these do not cover the head and cheeks only but are made to fit all round the faces of the riders with apertures for the eyes . . . From the helmets hang yellow plumes , a matter of décor as much as utility . As the horses move forward , the slightest breeze adds to the beauty of these plumes . — Arrian , Ars Tactica 34 \n"
3,"The M @-@ 6 Trail was constructed in a $ 3 @.@ 5 million project ( equivalent to $ 4 million in 2015 ) that started in 2008 . The goal was to create a 10 @-@ foot @-@ wide ( 3 @.@ 0 m ) path linking the Kent Trails with the Paul Henry Rail Trail . The M @-@ 6 Trail was the brainchild of Gaines Township Supervisor Don Hilton , Sr. He had pushed to have the path included in the original freeway construction and opened with the rest of the South Beltline . The trail project was funded by $ 2 @.@ 9 million ( equivalent to $ 3 @.@ 3 million in 2015 ) in federal grants and $ 300 @,@ 000 ( equivalent to $ 350 @,@ 000 in 2015 ) from the Frederik and Lena Meijer Foundation . The balance came from Kent County and the townships . Work on the trail was completed in November 2008 . \n"
4,"The fictional character Stingaree proved to be a prototype of a character Hornung used in a series of six short stories published in 1898 in Cassell 's Magazine , A. J. Raffles . The character was modelled on George Cecil Ives , a Cambridge @-@ educated criminologist and talented cricketer who , like Raffles , was a resident of the Albany , a gentlemen 's only residence in Mayfair . The first tale of the series "" In the Chains of Crime "" was published in June that year , titled "" The Ides of March "" . The stories were collected into one volume — with two additional tales — under the name The Amateur Cracksman , which was published the following year . Hornung used a narrative form similar to Doyle 's Sherlock Holmes stories , with Raffles and his partner @-@ in @-@ crime ( and former school fag ) Bunny Manders being the criminal counterparts to Holmes and Dr. Watson — although Rowland writes that Raffles and Manders "" were also fictionalized versions of Wilde and Bosie "" ( Wilde 's lover , Lord Alfred Douglas ) . — and he dedicated the stories to his brother @-@ in @-@ law : "" To A.C.D. This form of flattery "" . Doyle had warned against writing the stories , and reflected in his memoirs that "" there are few finer examples of short @-@ story writing in our language than these , though I confess I think they are rather dangerous in their suggestion . I told him so before he put pen to paper , and the result has , I fear , borne me out . You must not make the criminal the hero "" . The book was a popular and financial success , although some critics also echoed Doyle 's fears . The reviewer in The Spectator wrote that "" stern moralists "" would consider the book 's premise "" as a new , ingenious , artistic , but most reprehensible application of the crude principles involved in the old @-@ fashioned hero @-@ worship of Jack Sheppard and Dick Turpin "" . The book ends with Manders imprisoned and Raffles apparently dead , something that left The Spectator reviewer "" expressing [ their ] satisfaction that this audaciously entertaining volume is not issued in a cheap form . It

# **Causal Language model**

In [ ]:
# model_checkpoint = "distilgpt2"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilgpt2', use_fast = True)

In [ ]:
def tokenize_function(examples) :
  return tokenizer(examples['text'])

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=['text'])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset['train'][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
block_size = 128

In [ ]:
def group_texts(examples) : 
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
  # customize this part to your needs.

  total_length = (total_length // block_size) * block_size

  result = {
      k : [t[i : i+block_size] for i in range(0, total_length, block_size)] for k, t in concatenated_examples.items()
  }

  result['labels'] = result['input_ids'].copy()

  return result

In [ ]:
lm_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=4000,
    num_proc=4
    
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [ ]:
from transformers import Trainer, TrainingArguments

model_name = 'distilgpt2'.split('/')[-1]
training_arguments = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

trainer.train()

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()

# **Masked Language Model**